In [1]:
import qcodes
from qdev_wrappers.station_configurator import StationConfigurator

from qdev_wrappers.sweep_functions import do1d, do2d, do0d

/Users/sschaal/anaconda3/envs/qcodes/lib/python3.6/site-packages/qdev_wrappers/logger.py:16: UserWarning: The logger.py of qdev-wrappers is deprecated and will be removed soon. Please use the logger of QCoDeS instead.
  warnings.warn('The logger.py of qdev-wrappers is deprecated and will be '
/Users/sschaal/anaconda3/envs/qcodes/lib/python3.6/site-packages/qdev_wrappers/station_configurator.py:21: UserWarning: ruamel yaml not found station configurator is falling back to pyyaml. It's highly recommended to install ruamel.yaml. This fixes issues with scientific notation and duplicate instruments in the YAML file
  warnings.warn("ruamel yaml not found station configurator is falling back to pyyaml. "


## QCodes monitor
The QCodes monitor provides an overview over selected parameters and their current values in a separate webpage. The monitor runs in a separate python shell and the snippet below shows how to run a separate shell in the background in a jupyter notebook cell using the \%\%script notebook magic command. Once an instance of the monitor class is created, where parameters that should be monitored are passed within the constructor, the monitor web-page should be filled with current parameter values.

In [ ]:
%%script cmd --bg
python -m qcodes.monitor.monitor

In [ ]:
monitor = qcodes.Monitor(param1, param2, param3, ...)

## Stepping
Large jumps in voltage might be harmful for a nano-scale device. The step and inter\_delay attribute of a parameter allows to define a maximum step and an effective ramp rate. 

In [ ]:
vsd.step = 0.001
vsd.inter_delay = 0.3

## Station configurator
When using the station-configurator of the QDev-wrappers, both parameter monitoring and stepping can be defined within the yaml file. Individual setting of the attributes and creating an instance of the monitor class is unnecessary.

In [ ]:
instruments:
    VNA:
        driver: qcodes.instrument_drivers.Keysight.Keysight_E5071C
        type: Keysight_E5071C
        address: TCPIP0::169.254.71.72::inst0::INSTR
        enable_forced_reconnect: true
        parameters:
            timeout: {initial_value: 1000}
            start:  {initial_value: 500000000, monitor: true}
            stop:  {initial_value: 900000000, monitor: true}
    keith:
        driver: qcodes.instrument_drivers.tektronix.Keithley_2400 
        type: Keithley_2400 
        init:
            address: GPIB0::26::INSTR
            terminator: "\n"
        enable_forced_reconnect: true
        parameters:
            volt:
                monitor: true
                step: 0.1
                inter_delay: 0.3

Instruments can then be loaded via the station-configurator and simple measurements can be performed using the doNd convenience functions.

In [ ]:
STATION = qc.Station()
SC = StationConfigurator('setup.yaml', station = STATION)

VNA = SC.load_VNA()
plot, data = do1d(VNA.power, -40, -10, 31, 0.001, VNA.trace)

## Custom parameter
Often it can be useful to define a custom virtual parameter as a function of physical parameters. This can be easily done using the qcodes.Parameter class as shown at the example of a symmetric source-drain bias voltage below. To every parameter a validator can be assigned that allows only specific input values. 


In [ ]:
def set_Vsd(volt):
source.offset(volt/2)
drain.offset(-volt/2)

vsd = qcodes.Parameter('Vsd', label='Vsd', unit='V', set_cmd=set_Vsd,
                       vals=qcodes.utils.validators.Numbers(-50e-3, 50e-3))

vsd(1e-3)

## Widgets
Widgets within a jupyter notebook provide a simple way to manually change parameters by entering numbers into text field or using sliders. Below an example for creating a floating number text widget to manually tune the pump signal frequency of a Josephson parametric amplifier is shown. More widgets can be found at:
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html.


In [ ]:
import ipywidgets as widgets

pumpfreq=widgets.FloatText(
    value=pump.frequency(),
    description='Frequency: ',
    min=500e6,
    max=800e6,
    step=0.1e6
)

In [ ]:
widgets.interact(pump.frequency, value=pumpfreq)

## Video mode
While matplotlib is a great python library to produce production-quality graphs, generating and updating graphs is slow. Pyqtgraph is a scientific plotting graphics library that is embedded within the Qt framework. Genertaing graphs is not as simple as using matplotlib but pyqtgraphs can be embedded into any Qt application allowing to create graphical interfaces for measurements with very fast rendering. QCodes offers a QtPlot class for generation of simple pyqtgraphs based on qcodes datasets. Below code to realise video-mode measurements using the QtPlot class is presented at the example of fast reflectometry measurements. The code can be adapted to any other two-dimensional measurement and could be extended to realise a complete graphical measurement interface.

To begin, the fast reflectometry measurement is set-up and a QtPlot is generated from an initial reflectometry measurement.

In [ ]:
# Uses station-configurator
alazar=SC.load_Alazar()
awg=SC.load_AWG()
reflectometry=SC.load_AlazarFastReflectometry()

reflectometry.sample_rate(2e6)
# amplifier is already AC coupled
reflectometry.input_coupling('DC') 
reflectometry.input_range(0.8)
reflectometry.impedance(1000000)

# setup AWG voltage space for fast and slow ramp
reflectometry.y_start(0.53)
reflectometry.y_end(0.54)
reflectometry.y_npts(256)
reflectometry.x_start(-6e-3)
reflectometry.x_end(6e-3)
reflectometry.x_npts(256)

# enter I and Q DC offset for accurate calculation of phase in AC coupling mode
reflectometry.I_DC(6.4)
reflectometry.Q_DC(5.3)

reflectometry.awg=awg
# ramp to setpoint for x and y, and calculate frequency from sample rate and npts 
reflectometry.setup_AWG()
reflectometry.acquisition.setup_sweep(buffers_per_acquisition=10) # 10 averages

# create plot from one measurement
# alternatively use:
# plot, dataset = do0d(reflectometry.acquisition)
# which will create additional graphs for mag and phase
plot = qcodes.QtPlot()
dataset=qcodes.Measure(reflectometry.acquisition)
plot.add(dataset.Alazar_I)
plot.add(dataset.Alazar_Q)

Next, within a while-loop new measurement data is acquired and the selected graphs are updated. A waiting time between individual updates can be defined. To stop live updates the loop needs to be interrupted. 

In [ ]:
import time
wait = 0 # seconds

# select I and Q array to plot, choose 2,3 for mag and phase 
selection=[0,1]

# loop until interrupted
while True: 
    # returns tuple of I,Q,mag,phase data acquired in less than a second
    # data is not saved on the harddrive
    data=reflectometry.acquisition()
    # update graphs
    for i in range(len(selection)):
        po=plot.traces[i]['plot_object']
        po['image'].setImage(
          data[selection[i]].transpose(), 
          levels=po['hist'].getLevels()
        )
    time.sleep(wait)